# Coastal Shoreline Erosion Dynamic Model

This notebook implements the coastal erosion model strictly following
the documented governing equations:

\[
R = \frac{dR}{dt} = R_{wave} + R_{SLR} + R_{sediment}
\]

All formulations, symbols, and structures are kept unchanged.
No simplification or reformulation is introduced.


## Governing Equations

### Total erosion rate
\[
\frac{dR}{dt} = R_{wave} + R_{SLR} + R_{sediment}
\]

### Wave-induced erosion
\[
R_{wave}
=
- k_1
\frac{H_s^2 \cdot T \cdot [1 + \alpha F(t)]}
{R_{sediment} \cdot \Psi_{veg}(t)}
\]

### Sea-level-rise-induced erosion
\[
R_{SLR}
=
- k_2 \cdot
\frac{L}{B + h}
\cdot
\frac{dS}{dt}
\]

### Sediment-induced erosion
\[
R_{sediment}
=
k_3
\frac{\Delta Q(t)}{W}
\]


In [1]:
import numpy as np
from scipy.integrate import solve_ivp


In [11]:
import numpy as np
from scipy.interpolate import interp1d

# Hypothetical observation data
t_data = np.array([0, 10, 20, 30, 40, 50])

F_data = np.array([0.20, 0.30, 0.40, 0.50, 0.60, 0.70])
Psi_data = np.array([0.90, 0.85, 0.80, 0.75, 0.70, 0.65])
DeltaQ_data = np.array([1.50, 1.30, 1.10, 0.90, 0.80, 0.70])
dSdt_data = np.array([0.008, 0.009, 0.010, 0.011, 0.012, 0.013])

# Interpolation functions
F_func = interp1d(t_data, F_data, fill_value="extrapolate")
Psi_func = interp1d(t_data, Psi_data, fill_value="extrapolate")
DeltaQ_func = interp1d(t_data, DeltaQ_data, fill_value="extrapolate")
dSdt_func = interp1d(t_data, dSdt_data, fill_value="extrapolate")

def F(t):
    return float(F_func(t))

def Psi_veg(t):
    return float(Psi_func(t))

def DeltaQ(t):
    return float(DeltaQ_func(t))

def dS_dt(t):
    return float(dSdt_func(t))


In [12]:
# Empirical coefficients
k1 = 0.5
k2 = 0.3
k3 = 0.4
alpha = 0.2

# Physical parameters
Hs = 1.5     # mean significant wave height
T = 8.0      # wave period
L = 1000.0   # active profile length
B = 50.0     # beach width
h = 10.0     # closure depth
W = 200.0    # active shoreline width


In [13]:
def dR_dt(t, R):
    """
    dR/dt = R_wave + R_SLR + R_sediment
    """

    R_sediment = k3 * DeltaQ(t) / W

    R_wave = -k1 * (
        Hs**2 * T * (1 + alpha * F(t))
    ) / (R_sediment * Psi_veg(t))

    R_SLR = -k2 * (L / (B + h)) * dS_dt(t)

    return R_wave + R_SLR + R_sediment


In [14]:
t_span = (0, 50)        # time interval
t_eval = np.linspace(0, 50, 500)

R0 = [0.0]              # initial shoreline erosion state


In [15]:
solution = solve_ivp(
    dR_dt,
    t_span,
    R0,
    t_eval=t_eval,
    method='RK45'
)


In [17]:
solution.t, solution.y[0]


(array([ 0.        ,  0.1002004 ,  0.2004008 ,  0.3006012 ,  0.4008016 ,
         0.501002  ,  0.6012024 ,  0.70140281,  0.80160321,  0.90180361,
         1.00200401,  1.10220441,  1.20240481,  1.30260521,  1.40280561,
         1.50300601,  1.60320641,  1.70340681,  1.80360721,  1.90380762,
         2.00400802,  2.10420842,  2.20440882,  2.30460922,  2.40480962,
         2.50501002,  2.60521042,  2.70541082,  2.80561122,  2.90581162,
         3.00601202,  3.10621242,  3.20641283,  3.30661323,  3.40681363,
         3.50701403,  3.60721443,  3.70741483,  3.80761523,  3.90781563,
         4.00801603,  4.10821643,  4.20841683,  4.30861723,  4.40881764,
         4.50901804,  4.60921844,  4.70941884,  4.80961924,  4.90981964,
         5.01002004,  5.11022044,  5.21042084,  5.31062124,  5.41082164,
         5.51102204,  5.61122244,  5.71142285,  5.81162325,  5.91182365,
         6.01202405,  6.11222445,  6.21242485,  6.31262525,  6.41282565,
         6.51302605,  6.61322645,  6.71342685,  6.8

## Implementation Notes

- The model is implemented as a coupled nonlinear ordinary differential equation.
- The sediment-induced erosion term appears explicitly in the denominator of the wave-induced erosion term, preserving the original coupling structure.
- Time-dependent functions such as \(F(t)\), \(\Psi_{veg}(t)\), and \(\Delta Q(t)\) are treated as externally defined inputs.
- No mathematical simplification or structural modification has been applied.
